In [1]:
# import dependencies
import gmaps
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import api key
from api_keys import (wkey, gkey)

In [3]:
# configure gmaps
gmaps.configure(api_key = gkey)

In [4]:
# import the data compiled by WeatherPy
main_df = pd.read_csv("Output/city_weather_data.csv")

# preview the DataFrame
main_df.head()

,Country,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),DateTime
0,Indonesia,banda aceh,5.5577,95.3222,87.12,59,96,11.83,1660282166
1,Brazil,acarau,-2.8856,-40.1200,75.85,87,62,12.86,1660282167
2,Australia,esperance,-33.8667,121.9000,58.21,55,61,7.31,1660282169
3,Mauritius,mahebourg,-20.4081,57.7000,71.89,64,40,17.27,1660282170
4,Cape Verde,ponta do sol,17.1994,-25.0919,76.78,82,61,11.27,1660282172


### Humidity Heat Map

In [5]:
# get the location data
locations = main_df[["Latitude", "Longitude"]]

# get the Humidity data
humidity = main_df["Humidity (%)"].astype(float)

In [6]:
# initialize the map figure
mapFig = gmaps.figure()

# define the heat layer
heat_layer = gmaps.heatmap_layer(locations,
                                weights = humidity,
                                dissipating = False,
                                max_intensity = max(humidity),
                                point_radius = 3)

# add the heatmap layer
mapFig.add_layer(heat_layer)

# display the map figure
mapFig

Figure(layout=FigureLayout(height='420px'))

### Idealized Conditions

In [7]:
# define the ideal conditions
maxTemp = 80
minTemp = 70
maxWindSpeed = 10
targetCloudiness = 0

In [8]:
# filter out undesired conditions
idealized_df = main_df.loc[
                        (main_df["Max Temperature (F)"] <= maxTemp) & 
                        (main_df["Max Temperature (F)"] >= minTemp) &
                        (main_df["Wind Speed (mph)"] <= maxWindSpeed) &
                        (main_df["Cloudiness (%)"] == targetCloudiness)].reset_index(drop = True)

# show how many records match the provided criteria
currentCount = len(idealized_df)
print(f"There are {currentCount} cities that match the criteria.")

# limit the maximum number of ideal locations to 10
if currentCount > 10:
    idealized_df = idealized_df.iloc[0:10, :]
    print(f"{currentCount - 10} records had to be removed.")

# preview the DataFrame
idealized_df.head(len(idealized_df))

There are 16 cities that match the criteria.
6 records had to be removed.


,Country,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),DateTime
0,United States of America,cody,44.5263,-109.0565,76.01,65,0,0.00,1660282192
1,Brazil,natal,-5.7950,-35.2094,73.62,94,0,4.61,1660282168
2,United States of America,lexington park,38.2668,-76.4538,77.68,86,0,6.91,1660282155
3,Jordan,busayra,30.7376,35.6078,73.99,37,0,3.69,1660282446
4,Egypt,marsa matruh,31.3525,27.2453,75.40,73,0,6.91,1660282548
5,United States of America,hastings,40.5861,-98.3884,74.82,64,0,8.05,1660282464
6,Brazil,conceicao do araguaia,-8.2578,-49.2647,73.98,44,0,4.76,1660282646
7,French Guiana,cayenne,4.9333,-52.3333,78.10,1,0,0.00,1660282668
8,Russian Federation,kolyshley,52.7005,44.5367,72.97,66,0,7.16,1660282805
9,Libyan Arab Jamahiriya,bardiyah,31.7561,25.0865,78.69,69,0,8.72,1660282812


### Locate Nearby Hotels

In [9]:
# define the base url for Places API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# add column to the DataFrame
idealized_df["Hotel"] = ""

# iterate through the idealized locations
for index, row in idealized_df.iterrows():
    
    # define the API request parameters
    parameters = {
            "location": f"{row['Latitude']}, {row['Longitude']}",
            "keyword": "Hotel",
            "radius": 5000,
            "target_type": "lodging",
            "key": gkey}
    
    # perform the API request
    response_json = requests.get(base_url, params = parameters).json()
    
    # add the hotel names
    if response_json["status"] == "OK":
        idealized_df.at[index, "Hotel"] = response_json["results"][0]["name"]
    else:
        idealized_df.at[index, "Hotel"] = "None Supplied"
        print(f"{index}: ZERO RESULTS")

# drop rows that don't contain hotel names
idealized_df = idealized_df.loc[idealized_df["Hotel"] != "None Supplied", :]

# preview the updated DataFrame
idealized_df.head(15)

8: ZERO RESULTS


,Country,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),DateTime,Hotel
0,United States of America,cody,44.5263,-109.0565,76.01,65,0,0.00,1660282192,Hampton Inn & Suites Cody
1,Brazil,natal,-5.7950,-35.2094,73.62,94,0,4.61,1660282168,Wish Natal
2,United States of America,lexington park,38.2668,-76.4538,77.68,86,0,6.91,1660282155,Holiday Inn Express & Suites
3,Jordan,busayra,30.7376,35.6078,73.99,37,0,3.69,1660282446,Ein Lahda Camp Site
4,Egypt,marsa matruh,31.3525,27.2453,75.40,73,0,6.91,1660282548,Beau Site
5,United States of America,hastings,40.5861,-98.3884,74.82,64,0,8.05,1660282464,"Holiday Inn Express Hastings, an IHG Hotel"
6,Brazil,conceicao do araguaia,-8.2578,-49.2647,73.98,44,0,4.76,1660282646,Pousada Renascer
7,French Guiana,cayenne,4.9333,-52.3333,78.10,1,0,0.00,1660282668,Grand Hotel Montabo
9,Libyan Arab Jamahiriya,bardiyah,31.7561,25.0865,78.69,69,0,8.72,1660282812,Hotel Burdy


In [10]:
# redefine the locations
locations = idealized_df[["Latitude", "Longitude"]]

# construct the info box content template
template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# define the info box content
ibc = [template.format(**row) for index, row in idealized_df.iterrows()]

# # define the marker layer
marker_layer = gmaps.marker_layer(
                            locations,
                            info_box_content = ibc)

# add the marker layer
mapFig.add_layer(marker_layer)

# display the map figure
mapFig

Figure(layout=FigureLayout(height='420px'))